# Aula 4.1

## Conectando Google Colab ao Google Cloud

Para que o Google Colab possa acessá-los, precisamos nos autenticar. Isso garante que apenas você (e quem você der permissão) possa ver seus dados.


In [ ]:
from google.colab import auth

auth.authenticate_user()
# Inicia o processo de autenticação do usuário.

print('Autenticação no Google Cloud concluída! Agora podemos acessar nossos recursos.')

Autenticação no Google Cloud concluída! Agora podemos acessar nossos recursos.


## Lendo dados brutos direto do Cloud Storage (JSON feriados)

Primeiramente, vamos conectar o Colab ao GCS para acessar dados brutos, como o JSON dos feriados.

Vamos usar a biblioteca `google.cloud.storage` para ler o arquivo `feriados.json` que subimos na aula passada. Lembrem-se de substituir os nomes do projeto, bucket e caminho do arquivo pelos seus próprios!

In [ ]:
from google.cloud import storage

In [ ]:
# --- Configure seus detalhes aqui ---
project_id = 'alura-001111' # Ex: 'projeto-olist-analise-123'
bucket_name = 'fundamentos_nuvem' # Ex: 'dados-olist-joaousuario'
file_name = 'BR.json' # Caminho completo do arquivo dentro do bucket

In [ ]:
# Inicializa o cliente do Cloud Storage, usando o projeto autenticado
client_gcs = storage.Client(project=project_id)

bucket = client_gcs.bucket(bucket_name)

blob = bucket.blob(file_name)

In [ ]:
# Baixar o conteúdo do arquivo do GCS
file_content_str = blob.download_as_text()

# Exibir as primeiras 500 caracteres do conteúdo
print(f"Conteúdo do arquivo '{file_name}' (primeiras 500 caracteres):")
print(file_content_str)

Conteúdo do arquivo 'BR.json' (primeiras 500 caracteres):
[{"date":"2016-01-01","localName":"Confraternização Universal","name":"New Year's Day","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Public"]},{"date":"2016-02-08","localName":"Carnaval","name":"Carnival","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Bank","Optional"]},{"date":"2016-02-09","localName":"Carnaval","name":"Carnival","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":nu

...


In [ ]:
import json # Para lidar com o conteúdo JSON

# Carregar o conteúdo como um objeto JSON em Python
data_feriados_json = json.loads(file_content_str)

# Mostrar o primeiro item do JSON de feriados
print("\nPrimeiro item do JSON de feriados carregado em Python:")
print(data_feriados_json[0])  # Mostra o primeiro feriado como um dicionário Python


Primeiro item do JSON de feriados carregado em Python:
{'date': '2016-01-01', 'localName': 'Confraternização Universal', 'name': "New Year's Day", 'countryCode': 'BR', 'fixed': False, 'global': True, 'counties': None, 'launchYear': None, 'types': ['Public']}


# Aula 4.2 - Acessando dados do Bigquery com Python

Para trabalhar com nosso dataset Olist (que é maior e mais complexo) de forma eficiente, e fazer análises com SQL em escala, o BigQuery é a ferramenta principal. O Python, aqui, será nosso "responsável" que envia as queries para o BigQuery e pega os resultados.

Agora, vamos inicializar o cliente para nos conectar ao BigQuery. Ele usará a mesma autenticação que já fizemos.

In [ ]:
from google.cloud import bigquery

In [ ]:
client_bq = bigquery.Client(project=project_id)

print(f'\nCliente BigQuery conectado ao projeto: {client_bq.project}')


Cliente BigQuery conectado ao projeto: alura-465911


Vamos executar uma query SQL bem básica para inspecionar algumas linhas da nossa tabela de pedidos da Olist que está no BigQuery. O BigQuery fará o trabalho pesado de buscar os dados, e o Python apenas os exibirá.

In [ ]:
query_pedidos = """
SELECT
    order_id,
    order_status,
    order_purchase_timestamp,
    order_estimated_delivery_date,
    order_delivered_customer_date
FROM
    `alura-465911.olist_eccomerce.orders` -- Substitua pelo caminho completo da sua tabela de pedidos no BigQuery
"""

In [ ]:
query_job = client_bq.query(query_pedidos)

results = query_job.result()

In [ ]:
for row in results:
    print(f"ID Pedido: {row.order_id}, Status: {row.order_status}, Data Estimada: {row.order_estimated_delivery_date}")
    break

ID Pedido: a2e4c44360b4a57bdff22f3a4630c173, Status: approved, Data Estimada: 2017-03-01 00:00:00+00:00


In [ ]:
df_atraso = query_job.to_dataframe()
df_atraso

,order_id,order_status,order_purchase_timestamp,order_estimated_delivery_date,order_delivered_customer_date
0,a2e4c44360b4a57bdff22f3a4630c173,approved,2017-02-06 20:18:17+00:00,2017-03-01 00:00:00+00:00,NaT
1,132f1e724165a07f6362532bfb97486e,approved,2017-04-25 01:25:34+00:00,2017-05-22 00:00:00+00:00,NaT
2,809a282bbd5dbcabb6f2f724fca862ec,canceled,2016-09-13 15:24:19+00:00,2016-09-30 00:00:00+00:00,NaT
3,e5215415bb6f76fe3b7cb68103a0d1c0,canceled,2016-10-22 08:25:27+00:00,2016-10-24 00:00:00+00:00,NaT
4,71303d7e93b399f5bcd537d124c0bcfa,canceled,2016-10-02 22:07:52+00:00,2016-10-25 00:00:00+00:00,NaT
...,...,...,...,...,...
99436,4cccc0d35e7c7a0dc766ad3c4043e33e,unavailable,2018-08-10 09:32:32+00:00,2018-08-15 00:00:00+00:00,NaT
99437,897b4da63b6edde1a33a9fb7caf1dd10,unavailable,2018-07-30 07:38:21+00:00,2018-08-16 00:00:00+00:00,NaT
99438,93881917b8e0f2bf11eec7abbbfe43ec,unavailable,2018-08-11 21:38:00+00:00,2018-08-21 00:00:00+00:00,NaT
99439,4bd0d8aa4756f78245bd56015d4ddcc0,unavailable,2018-08-11 11:56:24+00:00,2018-08-27 00:00:00+00:00,NaT


In [ ]:
sql_atraso_combinado = f"""
SELECT
    order_id,
    order_estimated_delivery_date,
    order_delivered_customer_date,
    DATE_DIFF(order_delivered_customer_date, order_estimated_delivery_date, DAY) AS atraso_medio_dias
FROM
    `alura-465911.olist_eccomerce.orders`
WHERE
    order_delivered_customer_date IS NOT NULL
    AND order_estimated_delivery_date IS NOT NULL
    AND order_delivered_customer_date > order_estimated_delivery_date -- Filtra apenas os pedidos que atrasaram
ORDER BY
    atraso_medio_dias DESC
    LIMIT 10
"""

In [ ]:
results = client_bq.query(sql_atraso_combinado)
df_atraso = results.to_dataframe()
df_atraso

,order_id,order_estimated_delivery_date,order_delivered_customer_date,atraso_medio_dias
0,1b3190b2dfa9d789e1f14c05b647a14a,2018-03-15 00:00:00+00:00,2018-09-19 23:24:07+00:00,188
1,ca07593549f1816d26a572e06dc1eab6,2017-03-22 00:00:00+00:00,2017-09-19 14:36:39+00:00,181
2,47b40429ed8cce3aee9199792275433f,2018-01-19 00:00:00+00:00,2018-07-13 20:51:31+00:00,175
3,2fe324febf907e3ea3f2aa9650869fa5,2017-04-05 00:00:00+00:00,2017-09-19 17:00:07+00:00,167
4,285ab9426d6982034523a855f55a885e,2017-04-06 00:00:00+00:00,2017-09-19 14:00:04+00:00,166
5,440d0d17af552815d15a9e41abe49359,2017-04-07 00:00:00+00:00,2017-09-19 15:12:50+00:00,165
6,c27815f7e3dd0b926b58552628481575,2017-04-10 00:00:00+00:00,2017-09-19 17:14:25+00:00,162
7,0f4519c5f1c541ddec9f21b3bddd533a,2017-04-11 00:00:00+00:00,2017-09-19 14:38:21+00:00,161
8,d24e8541128cea179a11a65176e0a96f,2017-06-26 00:00:00+00:00,2017-12-04 18:36:29+00:00,161
9,2d7561026d542c8dbd8f0daeadf67a43,2017-04-13 00:00:00+00:00,2017-09-19 14:38:18+00:00,159


# 4.3. Carregando arquivos para o BigQuery e GCS

In [ ]:
# Inicializando o cliente do BigQuery
client = bigquery.Client(project="alura-000011")

# Definindo o ID do dataset e tabela de destino
dataset_id = 'alura-000011.olist_eccomerce'
table_id = f'{dataset_id}.orders_atrasos'

# Configurando o schema da tabela (opcional)
# Isso é necessário se você precisar definir os tipos de dados das colunas
schema = [
    bigquery.SchemaField("order_id", "STRING"),
    bigquery.SchemaField("order_estimated_delivery_date", "TIMESTAMP"),
    bigquery.SchemaField("order_delivered_customer_date", "TIMESTAMP"),
    bigquery.SchemaField("atraso_medio_dias", "INTEGER"),

]

# Enviando o DataFrame para o BigQuery
job_config = bigquery.LoadJobConfig(
    schema=schema,  # Usando schema se for necessário
    write_disposition="WRITE_APPEND",
)

# Carregando o DataFrame para o BigQuery
job = client.load_table_from_dataframe(df_atraso, table_id, job_config=job_config)

# Esperando a conclusão do job
job.result()  # Aguarda até que a operação de upload seja concluída

# Verificando se a tabela foi criada ou atualizada
print(f'Tabela {table_id} carregada com sucesso!')

## Carregando arquivos no GCS

In [ ]:

# Salve o DataFrame como um arquivo CSV
df_atraso.to_csv('dados.csv', index=False)

# Inicialize o cliente do Google Cloud Storage
client = storage.Client(project="alura-000111")

# Defina o nome do seu bucket e o caminho de destino
bucket_name = 'fundamentos_nuvem'
destination_blob_name = 'dados/dados.csv'

# Obtenha o bucket
bucket = client.get_bucket(bucket_name)

# Crie o blob (arquivo) e faça o upload do arquivo CSV
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename('dados.csv')

print(f'Arquivo enviado com sucesso para o GCS: gs://{bucket_name}/{destination_blob_name}')

Arquivo enviado com sucesso para o GCS: gs://fundamentos_nuvem/dados/dados.csv
